In [1]:
from main import rul_predictor

In [2]:
from preprocessing import *
from models import *

In [3]:
predictor = rul_predictor()
motor_name = "FD001"
iteration = "1"
SEQ_LENGTH = 20

predictor.read_data(filename_train='CMaps/train_' + motor_name + '.txt',
                    filename_test="CMaps/test_" + motor_name + ".txt",
                    filename_y="CMaps/RUL_" + motor_name + ".txt")
FEATURES = predictor.setting_names + predictor.sensor_names

predictor.train = predictor.add_RUL(predictor.train)
predictor.normalize(predictor.train, predictor.test, FEATURES)
predictor.train = predictor.cap_rul(predictor.train, 125)

In [4]:
compute_snr_for_sensors(predictor.train, FEATURES)

SNR for setting_1: 0.95
SNR for setting_2: 0.97
SNR for setting_3: inf
SNR for s_1: inf
SNR for s_2: 4.22
SNR for s_3: 3.47
SNR for s_4: 7.96
SNR for s_5: inf
SNR for s_6: 1.13
SNR for s_7: 7.50
SNR for s_8: 9.12
SNR for s_9: 36.63
SNR for s_10: inf
SNR for s_11: 11.19
SNR for s_12: 9.82
SNR for s_13: 9.22
SNR for s_14: 46.63
SNR for s_15: 5.43
SNR for s_16: inf
SNR for s_17: 4.03
SNR for s_18: inf
SNR for s_19: inf
SNR for s_20: 4.96
SNR for s_21: 5.02


{'setting_1': np.float64(0.9451877023176551),
 'setting_2': np.float64(0.970536446374189),
 'setting_3': inf,
 's_1': inf,
 's_2': np.float64(4.220975309512715),
 's_3': np.float64(3.4692773164925863),
 's_4': np.float64(7.96220151929913),
 's_5': inf,
 's_6': np.float64(1.125543257191568),
 's_7': np.float64(7.501102225047455),
 's_8': np.float64(9.12056429329275),
 's_9': np.float64(36.62965653339209),
 's_10': inf,
 's_11': np.float64(11.190856968332973),
 's_12': np.float64(9.819002519271947),
 's_13': np.float64(9.221744653103121),
 's_14': np.float64(46.62669096035619),
 's_15': np.float64(5.434626032354147),
 's_16': inf,
 's_17': np.float64(4.029056855129847),
 's_18': inf,
 's_19': inf,
 's_20': np.float64(4.955136432686272),
 's_21': np.float64(5.023901569102231)}

In [5]:
# predictor.train = apply_savgol_filter(predictor.train, FEATURES, 3, 1)

# predictor.train.head()

In [6]:
# compute_snr_for_sensors(predictor.train, FEATURES)

In [7]:
export_path = "Plots/" + motor_name + "_VarTrhld_" + iteration
# variance_threshold_analysis(predictor.train, 0.009, export_path)

# predictor.train, FEATURES = remove_low_variance_features(predictor.train, FEATURES, 0.009)

# predictor.train.head()

In [8]:
X, y = predictor.generate_window(predictor.train, SEQ_LENGTH, FEATURES)

In [9]:
# FLATTEN the sequence for Random Forest
X_flat = X.reshape(X.shape[0], -1)

In [10]:
X_test, y_test = predictor.prepare_test_data(predictor.test, predictor.y_test, FEATURES, SEQ_LENGTH)

In [11]:
X_test_flat = X_test.reshape(X_test.shape[0], -1)

In [13]:
TUNING_FITS = 30
K_FOLD = 5
# # Hyperparameter tuning
# search_rf = predictor.tune_model("rf", X_flat, y, TUNING_FITS, K_FOLD)
# search_xgb = predictor.tune_model("xgb", X_flat, y, TUNING_FITS, K_FOLD)
# search_svm = predictor.tune_model("svm", X_flat, y, TUNING_FITS, K_FOLD)
search_mlp = tune_model("mlp", X_flat, y, TUNING_FITS, K_FOLD)

# # For LSTM (expects 3D input)
# search_lstm = tune_lstm(X, y, TUNING_FITS, 1, 100, motor_name, iteration)

# # For CNN (expects 3D input)
# search_cnn = tune_cnn(X, y, TUNING_FITS, 1, 100, motor_name, iteration)

# For Deep-CNN (expects 3D input)
# search_deep_cnn = tune_deep_cnn(X, y, TUNING_FITS, 1, 100, motor_name, iteration)

ValueError: can only convert an array of size 1 to a Python scalar

In [ ]:
search_rf = load_tuned_model("search_rf_object", "search_results/" + motor_name + "/Iteration" + iteration + "/")
search_xgb = load_tuned_model("search_xgb_object", "search_results/" + motor_name + "/Iteration" + iteration + "/")
search_svm = load_tuned_model("search_svm_object", "search_results/" + motor_name + "/Iteration" + iteration + "/")
search_mlp = load_tuned_model("search_mlp_object", "search_results/" + motor_name + "/Iteration" + iteration + "/")

In [ ]:
# # RF: Get the best model from it
best_rf_model = search_rf.best_estimator_
rf_params = best_rf_model.get_params()

rf_model, y_pred_rf = train_and_evaluate_rf(X_flat, y, X_test_flat, y_test, rf_params)

In [ ]:
# XGB: Get the best model from it
best_xgb_model = search_xgb.best_estimator_
xgb_params = best_xgb_model.get_params()
 
xgb_model, y_pred_xgb = train_and_evaluate_xgboost(X_flat, y, X_test_flat, y_test, xgb_params)

In [ ]:
# MLP: Get the best model from it
best_mlp_model = search_mlp.best_estimator_
mlp_params = best_mlp_model.get_params()

mlp_model, y_pred_mlp = train_and_evaluate_mlp(X_flat, y, X_test_flat, y_test, mlp_params)

In [ ]:
# SVM: Get the best model from it
best_svm_model = search_svm.best_estimator_
svm_params = best_svm_model.get_params()

svm_model, y_pred_svm = train_and_evaluate_svm(X_flat, y, X_test_flat, y_test, svm_params, 1)

In [ ]:
#LSTM NEW IMPLEMENTATION
best_lstm_hp = load_best_hyperparameters("tuner_results/" + motor_name + "/Iteration" + iteration + "/lstm_bayes_tuner", "rul_lstm_bayes")

lstm_model, y_pred_lstm = train_and_evaluate_lstm(X, y, X_test, y_test, best_lstm_hp, build_lstm_model, 100)

In [ ]:
#CNN NEW IMPLEMENTATION
best_cnn_hp = load_best_hyperparameters("tuner_results/" + motor_name + "/Iteration" + iteration + "/cnn_bayes_tuner", "rul_cnn_bayes")

cnn_model, y_pred_cnn = train_and_evaluate_cnn(X, y, X_test, y_test, best_cnn_hp, build_cnn_model, 100)

In [ ]:
#DEEP CNN NEW IMPLEMENTATION
best_deep_cnn_hp = load_best_hyperparameters("tuner_results/" + motor_name + "/Iteration" + iteration + "/deep_cnn_bayes_tuner", "rul_deep_cnn")

deep_cnn_model, y_pred_deep_cnn = train_and_evaluate_deep_cnn(X, y, X_test, y_test, best_deep_cnn_hp, build_deep_cnn, 100)

In [ ]:
from evaluation import *
predictions = {
    "rf": y_pred_rf,
    "xgb": y_pred_xgb,
    "svm": y_pred_svm,
    "mlp": y_pred_mlp,
    "lstm": y_pred_lstm,
    "cnn": y_pred_cnn,
    "Deep-CNN": y_pred_deep_cnn
}
save_path = "Plots/" + motor_name + "_PerfComp_It_" + iteration
plot_model_performance(y_test, predictions, save_path)

In [ ]:
import os
import pickle

def save_predictions_dict(predictions_dict, save_path="saved_predictions/predictions.pkl"):
    os.makedirs(os.path.dirname(save_path), exist_ok=True)
    with open(save_path, "wb") as f:
        pickle.dump(predictions_dict, f)
    print(f"💾 Saved predictions dictionary to: {save_path}")

def load_predictions_dict(load_path):
    with open(load_path, "rb") as f:
        predictions_dict = pickle.load(f)
    print(f"📂 Loaded predictions dictionary from: {load_path}")
    return predictions_dict

In [ ]:
save_predictions_dict(predictions, "y_preds/" + motor_name + "/Iteration" + iteration + "_y_preds.pkl")

In [ ]:
# predictions_dict = load_predictions_dict("y_preds/" + motor_name + "/Iteration" + iteration + "_y_preds.pkl")

In [ ]:
# save_path = "Plots/" + motor_name + "_PerfCompNEW_It_" + iteration

# plot_model_performance(y_test, predictions, save_path)